In [8]:
import fitz
import pandas as pd
import re

In [69]:
pdf_document = fitz.open("li.pdf")
toc = pdf_document.get_toc(simple=True)

In [70]:
pd.set_option('display.max_colwidth', 1000)
pd.set_option('display.max_rows', 1000)
pd.DataFrame(toc)

,0,1,2
0,1,1 Introduction,9
1,2,1.1 Calcium Imaging,10
2,3,1.1.1 Intracellular calcium dynamics and its modeling,11
3,3,1.1.2 Calcium indicators,13
4,3,1.1.3 Data analysis and its difficulties,15
5,2,1.2 Neural coding: from firing rate to correlation,16
6,3,1.2.1 Correlation quantification,17
7,2,1.3 Question statement,18
8,1,2 Materials and Methods,19
9,2,2.1 Software and hardware,19


In [71]:
def extract_image_from_page(page, page_number):
    image_list = page.get_images(full=True)
    if image_list is None:
        return None
    else:
        images_name = []
        info_names = []
        for img_index, img_info in enumerate(image_list):
            xref = img_info[0]
            image_info_name = img_info[7]
            base_image = pdf_document.extract_image(xref)
            image_bytes = base_image["image"]
            image_filename = f"image_{page_number+1}_{img_index + 1}.png"  # page number starts from 0 in fitz and 1 in pdf
            images_name.append(image_filename)
            info_names.append(image_info_name)
            with open(image_filename, "wb") as image_file:
                image_file.write(image_bytes)
        return images_name, info_names

In [72]:
def separate_header(text):
    # Define a regex pattern to match the modified header format
    header_pattern = r'^\d+\s.*\s\d+$'  # Matches lines with a number, text, number
    #add here other header regex patterns!!!
    
    # Find the header pattern in the text
    match = re.search(header_pattern, text, re.MULTILINE)
    
    if match:
        header = match.group(0)
        header_end_index = match.end()
        main_text = text[header_end_index:]
    else:
        # If no header pattern found, return the entire text as main text
        header = ''
        main_text = text
    
    return header, main_text

In [73]:
def find_element_by_title(data_list, title):
    for item in data_list:
        if title in item[4]:  # Check if the title is present in the 5th element (index 4) of the tuple
            return item
    return None 

In [74]:
parts_list = []

for i in range(len(toc)-1):
    title = toc[i][1]
    start_page = toc[i][2] -1
    
    if i != len(toc)-1:
        next_title = toc[i+1][1]
        next_title_start_page = toc[i+1][2] -1
        end_page = toc[i+1][2] -1
    else:
        end_page = len(pdf_document) -1  #!probelm of out of toc pages and generalization and end page

    part_text = []
    part_images = []
    for j in range(start_page, end_page+1): #page number in pdf start from 1 and in fitz from 0
        page = pdf_document.load_page(j)
        page_images, info_names = extract_image_from_page(page, j)

        #checking if image is in the part or belongs to previous or next part
        for k, l in zip(info_names, page_images): 
            image_bbox = page.get_image_bbox(k)[1] #y0
            page_blocks = page.get_text("blocks")
            if j == start_page:
                #title_block = list(filter(lambda x: title.strip() in x.strip(), page_blocks))
                title_block = find_element_by_title(page_blocks, title)
                title_bbox_y0 = title_block[1]
                if next_title_start_page == start_page:    
                    next_title_block = find_element_by_title(page_blocks, next_title)
                    next_title_bbox_y0 = next_title_block[1]
                    if image_bbox > title_bbox_y0 and image_bbox < next_title_bbox_y0:
                        part_images.append(l)
                    elif image_bbox > title_bbox_y0:
                        part_images.append(l)
            elif j == end_page:
                if next_title_start_page == end_page:
                    next_title_block = find_element_by_title(page_blocks, next_title)
                    next_title_bbox_y0 = next_title_block[1]
                    if image_bbox < next_title_bbox_y0:
                        part_images.append(l)
            else:
                part_images.append(l)

        text = page.get_text("text")
        page_header, page_text = separate_header(text)
        part_text.append(page_text)   #incluse index of characers
    
    if (i != len(toc)-1):   #!probelm of out of toc pages and generalization and end page
        part_text[0] = part_text[0].split(title)[1]   #to exclude text from previous part
        part_text[-1] = part_text[-1].split(next_title)[0]  #to exclude text from next part
    else:
        part_text[0] = part_text[0].split(title)[1]     
    '''
    if (i != len(toc)-1):   # Problem of out of toc pages and generalization and end page
        print(i, title, part_text)
    # Check if title is found before splitting to avoid IndexError
        if title in part_text[0]:
            part_text[0] = part_text[0].split(title)[1]   # to exclude text from previous part
        else:
            print(f"Warning: Title '{title}' not found in the beginning of its section.")
    
    # Check if next_title is found before splitting to avoid IndexError
        if next_title in part_text[-1]:
            part_text[-1] = part_text[-1].split(next_title)[0]  # to exclude text from next part
        else:
            print(f"Warning: Next title '{next_title}' not found at the end of the section.")
    else:
        if title in part_text[0]:
            part_text[0] = part_text[0].split(title)[1]
        else:
        # Handle case where title is not found for the last item in the toc
            print(f"Warning: Title '{title}' not found in the beginning of the last section.")
'''
    part_dict = {
        'title': title,
        'pages': f'{start_page+1}-{end_page+1}',  # page number starts from 0 in fitz and 1 in pdf
        'text': part_text,
        'images': part_images
    }

    parts_list.append(part_dict)

#pdf_document.close()

In [75]:
pd.set_option('display.max_colwidth', 10000)
pd.DataFrame(parts_list)

,title,pages,text,images
0,1 Introduction,9-10,"[\nRecording neural activity is critical for neuroscience research. Neural activity sculpts\nhow neurons transfer information and interact with each other. Different markers or\nindicators can mediate it. Nowadays, a variety of techniques have been developed to\nmeasure brain activity, such as single-channel recording, multi-channel recording, lo-\ncal field potentials (LFPs), electroencephalogram (EEG), magnetoencephalography\n(MEG), functional magnetic resonance imaging (fMRI), positron emission tomog-\nraphy (PET), calcium imaging, optical imaging of intrinsic signals (OIS), volage-\nsensitive dye imaging (VSDI). These techniques can be classified into two main cat-\negories: electrophysiology and imaging techniques. Electrophysiology (patch clamp,\nmulti-channel recording, LFPs, EEG, MEG) measures and manipulates neuronal\ndynamics such as spike trains, EPSC, EPSP, and LFP. Imaging techniques (fMRI,\nPET, calcium imaging, OIS, VSDI) can capture neuronal activities and identify\nstructures and connections at different scales, from cortex areas and neurons to\ndendrites and spines. Each technology has advantages and disadvantages regarding\ninvasiveness and temporal and spatial resolution. Therefore, combining two or more\ntechnologies to reveal the complex spatiotemporal patterns of neural activity from\na single neuron to the population level is common. Figure 1 gives an overview of\ntemporal resolution and spatial scale.\n, \nFigure 1:\nTemporal resolution and spatial scale of imaging and electrophysi-\nology techniques.\nThe mesoscopic scale is represented by the oval shaded area\n[1].\nINTRA: intracellular recording, SU/MU: single-unit/multi-unit recording,\nLFP: local field potential, MEA: multi-electrode array, VSDI: voltage-sensitive dye\nimaging, OIS: optical imaging of intrinsic signals, EEG/MEG: electroencephalog-\nraphy/magnetoencephalography, fMRI/PET: functional magnetic resonance imag-\ning/poistron emission tompgraphy.\n]",[image_10_1.png]
1,1.1 Calcium Imaging,10-11,"[\nAs one of the imaging techniques, calcium imaging has become more popular in the\nseveral decades. Neuronal spiking is assessed by measuring changes in fluorescence.\nCalcium imaging together with recombinant DNA technologies and microscopy is\na powerful tool in modulating and visualizing cellular and molecular interactions.\nIt can be used not only to observe neuronal activity but also to associate behav-\nioral characterization with physiological states. Under the field of view (FOV), it is\npossible to record the activities of hundreds of neurons. It also enables recording free-\nmoving animals in in-vivo experiments. Two-photon microscopy has excellent spatial\nresolution, but because the calcium signal is slow, two-photon calcium imaging has\nvery good spatial resolution but low temporal resolution. Although experiments’\nsetup and execution could be demanding, calcium imaging can be combined with\n, \nelectrophysiology to compensate the low temporal resolution. Figure 2 is an exam-\nple. Intra-cellular recording directly measures neuronal activities, usually taken as\nground truth. Calcium imaging records fluorescence signals as an indicator of neu-\nronal spiking activity, which is indirect. Fluorescence signals and neuronal spikes\ndiffer from each other. Figure 3 shows single action potential and its evoked calcium\nsignal, obtained from recording rats’ calyx of Held - a large synaptic terminal in\nbrainstem slice. We see action potential is quick (in ms) with a short decay, while\nfluorescence signal (in this case, Fura-2) is slow in several seconds with a longer\ndecay. The fluorescence signal is much noisier than the action potential. The cal-\ncium indicator Section 1.1.2 is one key point to facilitate functional imaging with\nalive animals. For example, with a cranial window over visual cortex of a transgenic\nmouse, it is able to image large scale of neuronal activities on layer 

In [15]:
max_title_length = max(len(toc[i][1]) for i in range(len(toc)))

for i in range(len(toc)):
    title = toc[i][1]  
    start_page = toc[i][2]
    if i != len(toc)-1:
        end_page = toc[i+1][2]
    else:
        end_page = len(pdf_document)

    print(f"{i:2}  {title:{max_title_length}}  {start_page:2}  {end_page:2}")


 0  1 Introduction                                          9  10
 1  1.1 Calcium Imaging                                    10  11
 2  1.1.1 Intracellular calcium dynamics and its modeling  11  13
 3  1.1.2 Calcium indicators                               13  15
 4  1.1.3 Data analysis and its difficulties               15  16
 5  1.2 Neural coding: from firing rate to correlation     16  17
 6  1.2.1 Correlation quantification                       17  18
 7  1.3 Question statement                                 18  19
 8  2 Materials and Methods                                19  19
 9  2.1 Software and hardware                              19  19
10  2.2 Fluorescence signal modeling                       19  20
11  2.2.1 Point process                                    20  22
12  2.2.2 Kernel                                           22  24
13  2.2.3 Exact integration as fast convolution            24  25
14  2.3 Signal characterization and correlation inference  25  25
15  2.3.1 

In [ ]:
def extract_image_from_page(page, page_number, title_no):
    image_list = page.get_images(full=True)
    if image_list is None:
        return None
    else:
        images_name = []
        for img_index, img_info in enumerate(image_list):
            image_coor_y1 = page.get_image_bbox(image_list[img_index])[1]
            title_coor_y1 = page.search(toc[title_no][1])[0][1]
            next_title_coor_y1 = page.search(toc[title_no+1][1])[0][1]

            if image_coor_y1 > title_coor_y1:
                if 


            xref = img_info[0]
            base_image = pdf_document.extract_image(xref)
            image_bytes = base_image["image"]
            image_filename = f"image_{page_number+1}_{img_index + 1}.png"  # page number starts from 0 in fitz and 1 in pdf
            images_name.append(image_filename)
            with open(image_filename, "wb") as image_file:
                image_file.write(image_bytes)
        return images_name

In [ ]:
test_page = pdf_document.load_page(10)
image_list = test_page.get_images(full=True)
#image_list
a = test_page.get_image_bbox(image_list[0])

for i, j in enumerate(image_list):
    print('iii:', i, 'jjj:', j)

a

In [ ]:
import fitz

pdf_document = fitz.open("li.pdf")
toc = pdf_document.get_toc(simple=True)
page_number = 9  # You mentioned you want to find coordinates on page 10
title_index = 1  # Replace 'i' with the actual index of the title you are interested in

# Load the specified page
page = pdf_document.load_page(page_number)

# Get the coordinates of the title from the TOC
title = toc[title_index][1]

# Find the text block on the page that corresponds to the title
text_block = page.search_for(title)

# Get the coordinates of the text block
title_coordinates = text_block[0]  # Assuming there is only one occurrence of the title on the page

# Now title_coordinates contains (x0, y0, x1, y1) of the title on the page
print("Title Coordinates:", title_coordinates)

# If you want to print the text block content as well, you can do:
title_text = page.get_text("text", clip=title_coordinates)
print("Title Text:", title_text)


In [ ]:
#following version tries not to search through all lines.
def separate_header(text):
    # Define a regex pattern to match the modified header format
    header_pattern = r'^\d+\s.*\s\d+$'  # Matches lines with a number, text, number
    
    # Find the header pattern at the beginning of the text
    match = re.match(header_pattern, text)
    
    if match:
        header = match.group(0)
        header_end_index = match.end()
        main_text = text[header_end_index:].strip()  # Remove header and any leading/trailing whitespace
    else:
        # If no header pattern found, return the entire text as main text
        header = ''
        main_text = text.strip()  # Remove any leading/trailing whitespace
    
    return header, main_text